In [10]:
###el bueno
import scipy.io
import numpy

import torchtext
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchtext.vocab import Vectors, GloVe
import torch.nn.functional as F
import pdb
from copy import deepcopy
import pdb

In [14]:
##FIZZ
num_constants = 7
#Background
zero_extension = torch.zeros(1,num_constants)
zero_extension[0,0] = 1
succ_extension = torch.eye(num_constants-1,num_constants-1)
succ_extension = torch.cat((torch.zeros(num_constants-1,1),succ_extension),1)
succ_extension = torch.cat((succ_extension,torch.zeros(1,num_constants)),0)


#Intensional Predicates
aux_extension = torch.zeros(num_constants,num_constants)
aux2_extension= torch.zeros(num_constants,num_constants)
target_extension = torch.zeros(1,num_constants)

valuation_init = [Variable(zero_extension), Variable(succ_extension), Variable(aux_extension), \
                  Variable(aux2_extension), Variable(target_extension)]

num_predicates= len(valuation_init)
intensional_predicates=[2,3,4]
num_intensional_predicates = len(intensional_predicates)

#Target
target = torch.zeros(num_constants,num_constants)
target_pos = [0,3,6]
for integer in target_pos:
    target[0,integer]=1 


In [15]:
print(valuation_init[1], target, valuation_init[3])

(Variable containing:
    0     1     0     0     0     0     0
    0     0     1     0     0     0     0
    0     0     0     1     0     0     0
    0     0     0     0     1     0     0
    0     0     0     0     0     1     0
    0     0     0     0     0     0     1
    0     0     0     0     0     0     0
[torch.FloatTensor of size 7x7]
, 
    1     0     0     1     0     0     1
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
[torch.FloatTensor of size 7x7]
, Variable containing:
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
[torch

In [12]:
def decoder_efficient(valuation, step, num_constants):
    
    ## Create valuation_new
    valuation_new = [deepcopy(valuation[0]),deepcopy(valuation[1]),Variable(torch.zeros(valuation[2].size())),Variable(torch.zeros(valuation[3].size())),Variable(torch.zeros(valuation[4].size()))]
                    
    ##Unifications
    rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
    rules_aux = rules_aux.repeat(num_predicates,1)
    embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)
    #embeddings_aux = F.dropout(embeddings_aux, p=.1,training=True)
    #rules_aux = F.dropout(rules_aux, p=.1, training=True)
    
    #unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
    
    unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
    unifs = torch.exp(-unifs)
    unifs_sum = torch.sum(unifs, 0)
    unifs= unifs/unifs_sum
    
    
    ###TODO I want to normalize across the rules!!
    #unifs = rbf(embeddings_aux, rules_aux).view(num_predicates,-1)
    #unifs = F.relu(unifs)
    #unifs_sum = torch.sum(unifs, 0)
    #unifs= unifs/unifs_sum
    #unifs = torch.min(unifs,Variable(torch.ones(unifs.size())))

    ##Get_Valuations
    for predicate in intensional_predicates:
        if valuation[predicate].size()[0] == 1:
            #pdb.set_trace()
            for s in range(num_constants):
                valuation_aux = Variable(torch.Tensor([0]))
                for body1 in range(num_predicates):
                    for body2 in range(num_predicates):
                        ## Get nums
                        if valuation[body1].size()[0] == 1:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][0,s],valuation[body2][0,s])
                            else: 
                                num = torch.min(valuation[body1][0,:],valuation[body2][:,s])
                                num = torch.max(num)
                        else:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][:,s],valuation[body2][0,s])
                                num = torch.max(num)
                            else:
                                num = torch.min(valuation[body1][s,:],valuation[body2][:,s])
                                #num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                                num = torch.max(num)

                        ## max across three rules
                        new = Variable(torch.Tensor([0]))
                        for rule in range(num_rules): 
                            unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                            new = torch.max(new,unif) 

                        num = num*new 
                        valuation_aux = torch.max(valuation_aux, num)
                        if valuation_aux.data[0] > 1:
                                pdb.set_trace()
                        if predicate == 3 and s==0 and num.data[0]>.3:
                            print('bodyyyyy0',body1, body2)
                        if predicate == 3 and s==2 and num.data[0]>.3:
                            print('bodyyyyy2',body1, body2)
                        if predicate == 3 and s==4 and num.data[0]>.3:
                            print('bodyyyyy4',body1, body2) 
                        #if predicate == 3 and s==0 and o==6 and num.data[0]>.3:
                            #print('bodyyyyy6',body1, body2)
                        #if predicate == 3 and s==0 and o==8 and num.data[0]>.3:
                            #print('bodyyyyy8',body1, body2)   
                valuation_new[predicate][0,s] = torch.max(valuation[predicate][0,s], valuation_aux) 
                
        else:
            for s in range(num_constants):
                for o in range(num_constants):
                    valuation_aux = Variable(torch.Tensor([0]))
                    for body1 in range(num_predicates):
                        for body2 in range(num_predicates):
                            ## Get nums
                            if valuation[body1].size()[0] == 1:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][0,s],valuation[body2][0,o])
                                else: 
                                    num = torch.min(valuation[body1][0,s],valuation[body2][s,o])
                                    #num = torch.max(num)
                            else:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][s,o],valuation[body2][0,o])
                                    #num = torch.max(num)
                                else: 
                                    num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                                    num = torch.max(num)
                                    num_aux = torch.min(valuation[body1][s,o],valuation[body2][s,o])
                                    num = torch.max(num, num_aux)

                            ## max across three rules
                            new = Variable(torch.Tensor([0]))
                            for rule in range(num_rules): 
                                unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                                new = torch.max(new,unif)
                                #could be amalgamate

                            num = num*new 
                            valuation_aux = torch.max(valuation_aux, num)
                            if valuation_aux.data[0] > 1:
                                pdb.set_trace()
                            if predicate == 3 and s==0 and o==1 and num.data[0]>.3:
                                print('bodyyyyy1',body1, body2)
                            if predicate == 3 and s==0 and o==3 and num.data[0]>.3:
                                print('bodyyyyy3',body1, body2)
                            if predicate == 3 and s==0 and o==5 and num.data[0]>.3:
                                print('bodyyyyy5',body1, body2) 
                            #if predicate == 3 and s==0 and o==6 and num.data[0]>.3:
                                #print('bodyyyyy6',body1, body2)
                            #if predicate == 3 and s==0 and o==8 and num.data[0]>.3:
                                #print('bodyyyyy8',body1, body2)     
                    valuation_new[predicate][s,o] = torch.max(valuation[predicate][s,o], valuation_aux) 
                
    return valuation_new
           
def amalgamate(x,y):
    return x + y - x*y

def rbf(x,y):
    ans = (x-y)**2
    ans = torch.mean(ans,1)
    ans = torch.exp(-ans)
    return ans    
            
                

In [23]:
num_iters = 50
learning_rate = .1
steps = 4

num_feat=5
num_rules = 4


#embeddings = Variable(torch.rand(num_predicates, num_feat), requires_grad=True)
#rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
#optimizer = torch.optim.Adam([embeddings,rules],lr=learning_rate)

embeddings = Variable(torch.eye(num_predicates), requires_grad=True)
#rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
rule1 = torch.Tensor([0,0,0,0,1,1,0,0,0,0,1,0,0,0,0]).view(1,-1)
rule2 = torch.Tensor([0,0,0,0,1,0,0,0,0,1,0,0,1,0,0]).view(1,-1)
rule3 = torch.Tensor([0,0,1,0,0,0,1,0,0,0,0,0,0,1,0]).view(1,-1)
rule4 = torch.Tensor([0,0,0,1,0,0,1,0,0,0,0,1,0,0,0]).view(1,-1)
rules = Variable(torch.cat((rule1,rule2,rule3,rule4),0), requires_grad=True)

optimizer = torch.optim.Adam([rules, embeddings],lr=learning_rate)

criterion = torch.nn.BCELoss(size_average=False)


for i in range(num_iters):
    optimizer.zero_grad()
    
    valuation = valuation_init
    for step in range(steps):
        valuation = decoder_efficient(valuation, step, valuation[0].size()[1])
        print('step',step,'valuationend', valuation[-1]) 
    loss = criterion(valuation[-1][0,:],Variable(torch.Tensor(target[0,:])))
    
    print(i,'lossssssssssssssssssssssssssss',loss.data[0])
    loss.backward()
    optimizer.step()
    
    

('step', 0, 'valuationend', Variable containing:
 0.1303  0.0317  0.0000  0.0000  0.0000  0.0000  0.0000
[torch.FloatTensor of size 1x7]
)
('step', 1, 'valuationend', Variable containing:
 0.1303  0.0317  0.0041  0.0041  0.0000  0.0000  0.0000
[torch.FloatTensor of size 1x7]
)
('step', 2, 'valuationend', Variable containing:
 0.1303  0.0317  0.0041  0.0041  0.0022  0.0005  0.0005
[torch.FloatTensor of size 1x7]
)
('step', 3, 'valuationend', Variable containing:
 0.1303  0.0317  0.0041  0.0041  0.0022  0.0005  0.0005
[torch.FloatTensor of size 1x7]
)
(0, 'lossssssssssssssssssssssssssss', 15.094644546508789)
('step', 0, 'valuationend', Variable containing:
 0.1681  0.0499  0.0000  0.0000  0.0000  0.0000  0.0000
[torch.FloatTensor of size 1x7]
)
('step', 1, 'valuationend', Variable containing:
 0.1681  0.0499  0.0083  0.0083  0.0000  0.0000  0.0000
[torch.FloatTensor of size 1x7]
)
('step', 2, 'valuationend', Variable containing:
 0.1681  0.0499  0.0083  0.0083  0.0016  0.0015  0.0015
[to

KeyboardInterrupt: 

In [19]:
print(embeddings)
print(rules)

Variable containing:
 3.4203 -0.0564 -1.0249  1.1802 -2.0555
-2.0298  2.7322  0.8521 -1.9915 -0.9466
-1.0079 -0.3611  2.8882 -0.8807 -1.1760
-2.1271 -1.6934 -1.7009  3.2628 -2.0239
 0.9679  0.4662 -1.5694  0.7924  2.9261
[torch.FloatTensor of size 5x5]

Variable containing:

Columns 0 to 9 
-1.0213 -0.5340  2.9563 -0.8758 -1.2994 -1.3733  1.1411  2.2007 -1.0307 -1.2618
 0.9622  0.6158 -1.7210  0.9602  3.0796  3.4086 -0.0539 -0.9063  1.1271 -2.1264
 0.8249  0.5791 -1.5778  0.8828  2.8106  0.7012  0.5416 -1.4895  0.8290  2.9930
 1.3720  0.4164  0.2095  0.7402  0.0720  0.6443  0.4722  0.7404  1.2811  0.0743

Columns 10 to 14 
-2.1189  2.7852  0.9527 -2.0552 -1.0241
 3.3749 -0.0445 -1.0357  1.1450 -2.0264
-1.3508  1.0224  2.3613 -1.6480 -1.3243
 0.6501  0.6409  0.7542  0.2649  1.2358
[torch.FloatTensor of size 4x15]



In [21]:

rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
rules_aux = rules_aux.repeat(num_predicates,1)
embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)


#unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
#print('aaaaa',unifs)
#print('oooo',F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1))

unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
unifs = torch.exp(-unifs)
unifs_sum = torch.sum(unifs, 0)
unifs= unifs/unifs_sum
print('finaaaal',unifs)


('finaaaal', Variable containing:

Columns 0 to 9 
 0.0021  0.0042  0.0047  0.3938  0.0063  0.9907  0.0041  0.2408  0.0007  0.9909
 0.0187  0.0017  0.0021  0.0639  0.3242  0.0009  0.0022  0.0936  0.9766  0.0008
 0.9760  0.0014  0.0017  0.1549  0.6583  0.0022  0.0019  0.2877  0.0206  0.0020
 0.0019  0.0015  0.0018  0.0390  0.0052  0.0023  0.0017  0.0720  0.0006  0.0022
 0.0014  0.9913  0.9897  0.3484  0.0060  0.0039  0.9902  0.3060  0.0014  0.0040

Columns 10 to 11 
 0.0049  0.0895
 0.3222  0.1058
 0.6652  0.2130
 0.0033  0.0213
 0.0044  0.5703
[torch.FloatTensor of size 5x12]
)


In [ ]:

input =torch.randn(5,7)
print(input)
print(F.dropout(input, p=.2,training=True))